# Analyzing WeRateDogs data
## Description
In the following notebook, I will clean a given dataset complemented with WeLoveDogs twitter data.

## First Steps
Importing the necessary libraries and files.

In [2]:
import pandas as pd
import numpy as np
import requests
import os

df = pd.read_csv("twitter-archive-enhanced.csv")

## Gather

In this part, I will gather all the necessary data to carry the project. There are additional datasets that should be included in this project. I will start with the tweet image predictions file and follow with the Tweepy data that will be stored in a JSON file.

### Tweet Image Predictions


This file is hosted on Udacity's servers and should be downloaded programatically.

In [2]:
url = "https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv"

content = requests.get(url).text #Get the content from the URL above

In [3]:
file = "image-predictions.tsv"

with open(file, "w", encoding="utf8") as f: #Write the content from the URL above in a file
    f.write(content) 

In [4]:
df_img = pd.read_csv("image-predictions.tsv", sep="\t")

### Tweepy Data

To retrieve this data, I will have to use the Twitter API and store the piles in a JSON file.

In [5]:
import tweepy
import json

consumer_key = ''
consumer_secret = ''
access_token = ''
access_secret = ''

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth)

In [6]:
id_list = df["tweet_id"].tolist() #Make a list out of all observations in "tweet_id" column.

With the help of [Richard Chadwick from Towards Data Science](https://towardsdatascience.com/tweepy-for-beginners-24baf21f2c25), I have come up with the following code to retrieve all tweets by ID and store its details in a json file.

The code above has been looking through all the observations in the "tweet_id" column of the main dataframe (df). However, a lot of the tweets have been deleted since 2017. In fact, as we can see below, 1193 tweets from the main dataframe have been deleting since 2017. Almost 50% of them.

In [7]:
with open("tweet_json_error_log.txt") as f:
    count = sum(1 for _ in f)

count #Count the number of errors when getting the tweets via API

1193

After building the json file, I will create the dataframe "df_json" out of that json file:

In [5]:
df_json = pd.read_json("tweet_json.txt", lines=True)

## Assess

Once all the data is gathered, I will procceed to assess the data and do a first visual approach of the data to see what can we clean.

In [4]:
df.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

In [5]:
df_img.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [6]:
df_img.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  2075 non-null   int64  
 1   jpg_url   2075 non-null   object 
 2   img_num   2075 non-null   int64  
 3   p1        2075 non-null   object 
 4   p1_conf   2075 non-null   float64
 5   p1_dog    2075 non-null   bool   
 6   p2        2075 non-null   object 
 7   p2_conf   2075 non-null   float64
 8   p2_dog    2075 non-null   bool   
 9   p3        2075 non-null   object 
 10  p3_conf   2075 non-null   float64
 11  p3_dog    2075 non-null   bool   
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [7]:
df_json.head()

,created_at,id,id_str,text,truncated,entities,extended_entities,source,in_reply_to_status_id,in_reply_to_status_id_str,...,favorite_count,favorited,retweeted,possibly_sensitive,possibly_sensitive_appealable,lang,retweeted_status,quoted_status_id,quoted_status_id_str,quoted_status
0,2017-08-01 16:23:56+00:00,892420643555336193,892420643555336192,This is Phineas. He's a mystical boy. Only eve...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 892420639486877696, 'id_str'...","<a href=""http://twitter.com/download/iphone"" r...",NaN,NaN,...,34689,False,False,0.0,0.0,en,NaN,NaN,NaN,NaN
1,2017-08-01 00:17:27+00:00,892177421306343426,892177421306343424,This is Tilly. She's just checking pup on you....,True,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,"<a href=""http://twitter.com/download/iphone"" r...",NaN,NaN,...,30053,False,False,0.0,0.0,en,NaN,NaN,NaN,NaN
2,2017-07-31 00:18:03+00:00,891815181378084864,891815181378084864,This is Archie. He is a rare Norwegian Pouncin...,True,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,"<a href=""http://twitter.com/download/iphone"" r...",NaN,NaN,...,22625,False,False,0.0,0.0,en,NaN,NaN,NaN,NaN
3,2017-07-30 15:58:51+00:00,891689557279858688,891689557279858688,This is Darla. She commenced a snooze mid meal...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 891689552724799489, 'id_str'...","<a href=""http://twitter.com/download/iphone"" r...",NaN,NaN,...,37963,False,False,0.0,0.0,en,NaN,NaN,NaN,NaN
4,2017-07-29 16:00:24+00:00,891327558926688256,891327558926688256,This is Franklin. He would like you to stop ca...,True,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,"<a href=""http://twitter.com/download/iphone"" r...",NaN,NaN,...,36243,False,False,0.0,0.0,en,NaN,NaN,NaN,NaN


In [8]:
df_json.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1163 entries, 0 to 1162
Data columns (total 30 columns):
 #   Column                         Non-Null Count  Dtype              
---  ------                         --------------  -----              
 0   created_at                     1163 non-null   datetime64[ns, UTC]
 1   id                             1163 non-null   int64              
 2   id_str                         1163 non-null   int64              
 3   text                           1163 non-null   object             
 4   truncated                      1163 non-null   bool               
 5   entities                       1163 non-null   object             
 6   extended_entities              834 non-null    object             
 7   source                         1163 non-null   object             
 8   in_reply_to_status_id          35 non-null     float64            
 9   in_reply_to_status_id_str      35 non-null     float64            
 10  in_reply_to_user_id     

### Quality
#### `df` table:
- in_reply_to_status_id, in_reply_to_user_id, retweeted_status_id and retweeted_status_user_id are formatted as floats. They should be integers.
- Missing expanded_urls values. 
- Data in doggo, floofer, pupper and puppo is not formatted correctly. Data should be categorical and merged into one column.
- Retweets are not useful for this project. Remove them. 

#### `df_img` table:
- Unstandarized data in p1, p2 and p3. The variables are upper and lower case (i.e "miniature_pinscher" and "German_shepherd".
- Character "_" instead of a blank space. 

#### `df_json`table:
- id_str should be formatted as string, not integer.
- in_reply_to_status_id and in_reply_to_user_id should be formatted as integers.
- in_reply_to_status_id_str ad in_reply_to_user_id_str should be formatted as strings.
- Lots of missing values in variables geo, coordinates, place and contributors.
- possibly_sensitive should be a bool value.
- Remove retweets.

### Tidiness

- `df_json`table should be part of `df` table.
- Lots of duplicated values in `df_json` table that are already contained in `df` table.

## Clean

In [58]:
df_clean = df.copy()
df_img_clean = df_img.copy()
df_json_clean = df_json.copy()

### Missing data

#### `df_json`: missing values in variables geo, coordinates, place and contributors.

#### Define

A lot of the values are missing for this variables. As it is impossible to fill up with the correct information, I will drop the columns.

#### Code

In [137]:
df_json_clean.drop(["geo", "coordinates", "place", "contributors"], axis=1, inplace=True)

#### Test

In [138]:
df_json_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1163 entries, 0 to 1162
Data columns (total 26 columns):
 #   Column                         Non-Null Count  Dtype              
---  ------                         --------------  -----              
 0   created_at                     1163 non-null   datetime64[ns, UTC]
 1   id                             1163 non-null   int64              
 2   id_str                         1163 non-null   int64              
 3   text                           1163 non-null   object             
 4   truncated                      1163 non-null   bool               
 5   entities                       1163 non-null   object             
 6   extended_entities              834 non-null    object             
 7   source                         1163 non-null   object             
 8   in_reply_to_status_id          35 non-null     float64            
 9   in_reply_to_status_id_str      35 non-null     float64            
 10  in_reply_to_user_id     

## Quality

#### `df_json` and `df`: remove retweets.

#### Define

Retweets are not useful for this analysis. I will remove them and then drop the columns.

#### Code

In [139]:
df_clean = df_clean[df_clean["retweeted_status_id"].isnull()] #Remove all non-null values (retweets)
df_json_clean = df_json_clean[df_json_clean["retweeted_status"].isnull()]

In [140]:
df_clean.drop(["retweeted_status_id", "retweeted_status_user_id", "retweeted_status_timestamp"], axis=1, inplace=True)
df_json_clean.drop("retweeted_status", axis=1, inplace=True)

#### Test

In [141]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175 entries, 0 to 2355
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   tweet_id               2175 non-null   int64  
 1   in_reply_to_status_id  78 non-null     float64
 2   in_reply_to_user_id    78 non-null     float64
 3   timestamp              2175 non-null   object 
 4   source                 2175 non-null   object 
 5   text                   2175 non-null   object 
 6   expanded_urls          2117 non-null   object 
 7   rating_numerator       2175 non-null   int64  
 8   rating_denominator     2175 non-null   int64  
 9   name                   2175 non-null   object 
 10  doggo                  2175 non-null   object 
 11  floofer                2175 non-null   object 
 12  pupper                 2175 non-null   object 
 13  puppo                  2175 non-null   object 
dtypes: float64(2), int64(3), object(9)
memory usage: 254.9+ 

In [142]:
df_json_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1059 entries, 0 to 1162
Data columns (total 25 columns):
 #   Column                         Non-Null Count  Dtype              
---  ------                         --------------  -----              
 0   created_at                     1059 non-null   datetime64[ns, UTC]
 1   id                             1059 non-null   int64              
 2   id_str                         1059 non-null   int64              
 3   text                           1059 non-null   object             
 4   truncated                      1059 non-null   bool               
 5   entities                       1059 non-null   object             
 6   extended_entities              790 non-null    object             
 7   source                         1059 non-null   object             
 8   in_reply_to_status_id          35 non-null     float64            
 9   in_reply_to_status_id_str      35 non-null     float64            
 10  in_reply_to_user_id     

#### `df`: combine doggo, floofer, pupper and puppo into one variable.

#### Define

It does not make sense to have four different categorical values separated. I will use the melt function to merge them into one simple variable.

#### Code

In [59]:
df_clean["term"] = df["doggo"] + df["floofer"] + df["pupper"] + df["puppo"]
df_clean["term"] = df_clean.term.str.extract(r'(doggo|floofer|pupper|puppo)')
df_clean.drop(["doggo", "floofer", "pupper", "puppo"], axis=1, inplace=True)

#### Test

In [60]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 14 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  term                        380 n

In [246]:
df_clean.alias.value_counts()

None       1831
pupper      224
doggo        87
puppo        24
floofer       9
Name: alias, dtype: int64

#### `df_json`: missing values in variables geo, coordinates, place and contributors.

#### Define

#### Code

#### Test

#### `df_json`: missing values in variables geo, coordinates, place and contributors.

#### Define

#### Code

#### Test

#### `df_json`: missing values in variables geo, coordinates, place and contributors.

#### Define

#### Code

#### Test

#### `df_json`: missing values in variables geo, coordinates, place and contributors.

#### Define

#### Code

#### Test

#### `df_json`: missing values in variables geo, coordinates, place and contributors.

#### Define

#### Code

#### Test

#### `df_json`: missing values in variables geo, coordinates, place and contributors.

#### Define

#### Code

#### Test

#### `df_json`: missing values in variables geo, coordinates, place and contributors.

#### Define

#### Code

#### Test

#### `df_json`: missing values in variables geo, coordinates, place and contributors.

#### Define

#### Code

#### Test

#### `df_json`: missing values in variables geo, coordinates, place and contributors.

#### Define

#### Code

#### Test

#### `df_json`: missing values in variables geo, coordinates, place and contributors.

#### Define

#### Code

#### Test

#### `df_json`: missing values in variables geo, coordinates, place and contributors.

#### Define

#### Code

#### Test

#### `df_json`: missing values in variables geo, coordinates, place and contributors.

#### Define

#### Code

#### Test

#### `df_json`: missing values in variables geo, coordinates, place and contributors.

#### Define

#### Code

#### Test

#### `df_json`: missing values in variables geo, coordinates, place and contributors.

#### Define

#### Code

#### Test

#### `df_json`: missing values in variables geo, coordinates, place and contributors.

#### Define

#### Code

#### Test

#### `df_json`: missing values in variables geo, coordinates, place and contributors.

#### Define

#### Code

#### Test

#### `df_json`: missing values in variables geo, coordinates, place and contributors.

#### Define

#### Code

#### Test

#### `df_json`: missing values in variables geo, coordinates, place and contributors.

#### Define

#### Code

#### Test

#### `df_json`: missing values in variables geo, coordinates, place and contributors.

#### Define

#### Code

#### Test

#### `df_json`: missing values in variables geo, coordinates, place and contributors.

#### Define

#### Code

#### Test

#### `df_json`: missing values in variables geo, coordinates, place and contributors.

#### Define

#### Code

#### Test

#### `df_json`: missing values in variables geo, coordinates, place and contributors.

#### Define

#### Code

#### Test

#### `df_json`: missing values in variables geo, coordinates, place and contributors.

#### Define

#### Code

#### Test

#### `df_json`: missing values in variables geo, coordinates, place and contributors.

#### Define

#### Code

#### Test

#### `df_json`: missing values in variables geo, coordinates, place and contributors.

#### Define

#### Code

#### Test

#### `df_json`: missing values in variables geo, coordinates, place and contributors.

#### Define

#### Code

#### Test

#### `df_json`: missing values in variables geo, coordinates, place and contributors.

#### Define

#### Code

#### Test

#### `df_json`: missing values in variables geo, coordinates, place and contributors.

#### Define

#### Code

#### Test

#### `df_json`: missing values in variables geo, coordinates, place and contributors.

#### Define

#### Code

#### Test

#### `df_json`: missing values in variables geo, coordinates, place and contributors.

#### Define

#### Code

#### Test

#### `df_json`: missing values in variables geo, coordinates, place and contributors.

#### Define

#### Code

#### Test

#### `df_json`: missing values in variables geo, coordinates, place and contributors.

#### Define

#### Code

#### Test

#### `df_json`: missing values in variables geo, coordinates, place and contributors.

#### Define

#### Code

#### Test